
이미지 총 1250  
train : 1000  
valid : 125  
test : 125  
  
시작 전 한번 실행  
> poetry install  
  
_path : data 경로



In [111]:
import numpy as np
import os
from sklearn.model_selection import StratifiedGroupKFold
import pandas as pd
from tqdm import tqdm
import shutil
from glob import glob

In [112]:
_path = './data'
test_name = 'valid'
if 'test' in os.listdir(_path):
    test_name = 'test'

print(test_name)


test


In [113]:
train_abnormal_file = os.path.join(_path, 'train-abnormal.csv')
train_acl_file = os.path.join(_path, "train-acl.csv")
train_meniscus_file = os.path.join(_path, "train-meniscus.csv")

test_abnormal_file = os.path.join(_path, f"{test_name}-abnormal.csv")
test_acl_file = os.path.join(_path, f"{test_name}-acl.csv")
test_meniscus_file = os.path.join(_path, f"{test_name}-meniscus.csv")
test_abnormal_file

train_abnormal_df = pd.read_csv(train_abnormal_file, names=['name', 'abnormal'])
train_acl_df = pd.read_csv(train_acl_file, names=['name', 'acl'])
train_meniscus_df = pd.read_csv(train_meniscus_file, names=['name', 'meniscus'])

test_abnormal_df = pd.read_csv(test_abnormal_file, names=['name', 'abnormal'])
test_acl_df = pd.read_csv(test_acl_file, names=['name', 'acl'])
test_meniscus_df = pd.read_csv(test_meniscus_file, names=['name', 'meniscus'])

abnormal_df  = pd.concat([train_abnormal_df, test_abnormal_df], axis=0)
acl_df  = pd.concat([train_acl_df, test_acl_df], axis=0)
meniscus_df  = pd.concat([train_meniscus_df, test_meniscus_df], axis=0)

df = pd.concat([abnormal_df, acl_df['acl'],meniscus_df['meniscus']], axis=1).sort_values('name')
df


,name,abnormal,acl,meniscus
0,0,1,0,0
1,1,1,1,1
2,2,1,0,0
3,3,1,0,1
4,4,1,0,0
...,...,...,...,...
1121,1245,1,1,1
1122,1246,1,1,1
1123,1247,1,0,1
124,1248,1,1,1


In [114]:
classes = {0 : [0,0,0], 1 : [0,0,1], 2 : [0,1,0], 3 : [0,1,1], 4 : [1,0,0], 5 : [1,0,1], 6 : [1,1,0], 7 : [1,1,1]}
# [abnormal, acl, meniscus]

df['class'] = [[k for k, v in classes.items() 
                        if v == [df['abnormal'].iloc[i], df['acl'].iloc[i], df['meniscus'].iloc[i]]][0]
                        for i in range(df.shape[0])]
df

,name,abnormal,acl,meniscus,class
0,0,1,0,0,4
1,1,1,1,1,7
2,2,1,0,0,4
3,3,1,0,1,5
4,4,1,0,0,4
...,...,...,...,...,...
1121,1245,1,1,1,7
1122,1246,1,1,1,7
1123,1247,1,0,1,5
124,1248,1,1,1,7


In [115]:
all = df.to_numpy()
X = np.ones(all.shape[0])
y = np.array([v[-1] for v in all])
groups = np.array([v[0] for v in all])
# groups
seed = 7777
fold_num = 6 # 0 ~ 9

cv = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=seed)

train_fold = []
test_fold = []

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])
    train_fold.append(groups[train_idx])
    test_fold.append(groups[val_idx])

test_fold[fold_num].shape

TRAIN: [   0    2    3 ... 1247 1248 1249]
  [4 4 5 ... 5 7 5]
 TEST: [   1   10   26   32   39   40   56   71   84   87  100  114  116  129
  136  154  165  172  177  190  207  213  222  230  243  251  258  268
  276  286  295  317  322  326  332  346  361  369  376  389  397  409
  428  432  440  450  462  470  476  489  502  507  518  531  545  555
  559  571  577  586  595  617  624  627  638  651  662  671  682  688
  698  709  723  730  744  745  755  769  781  792  801  816  818  829
  838  850  866  867  877  894  902  906  923  933  939  947  956  967
  978  993 1001 1005 1018 1028 1042 1053 1056 1067 1086 1087 1099 1113
 1124 1128 1144 1147 1157 1168 1183 1192 1206 1207 1219 1232 1243]
  [7 5 0 5 5 0 4 6 5 6 4 4 5 4 4 5 4 4 0 4 4 4 0 0 0 5 0 0 4 6 4 6 6 5 5 4 5
 4 4 4 5 0 0 5 4 0 0 5 0 5 5 5 5 5 4 6 6 0 5 5 4 5 7 4 4 6 5 0 4 7 5 4 7 4
 0 6 4 5 6 6 6 4 0 4 4 6 0 0 4 0 5 4 5 6 4 0 4 4 7 5 7 0 4 5 4 5 5 4 0 4 4
 4 4 4 0 0 4 4 6 7 6 7 6 5 5]
TRAIN: [   0    1    3 ... 1247 1248 1

(125,)

In [116]:
# check distribution
from collections import Counter
import pandas as pd

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0 
    
    distrs.append(get_distribution(train_y))
    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'test - fold{fold_ind}')

categories = [i for i in range(8)]
pd.DataFrame(distrs, index=index, columns = [categories[i] for i in range(np.max(y) + 1)])

,0,1,2,3,4,5,6,7
training set,19.36%,0.00%,0.00%,0.00%,36.24%,23.44%,8.48%,12.48%
train - fold0,19.29%,0.00%,0.00%,0.00%,36.44%,23.20%,7.91%,13.16%
test - fold0,20.00%,0.00%,0.00%,0.00%,34.40%,25.60%,13.60%,6.40%
train - fold1,19.02%,0.00%,0.00%,0.00%,36.53%,23.38%,8.44%,12.62%
test - fold1,22.40%,0.00%,0.00%,0.00%,33.60%,24.00%,8.80%,11.20%
train - fold2,19.38%,0.00%,0.00%,0.00%,35.73%,23.64%,8.44%,12.80%
test - fold2,19.20%,0.00%,0.00%,0.00%,40.80%,21.60%,8.80%,9.60%
train - fold3,19.91%,0.00%,0.00%,0.00%,36.36%,23.29%,8.00%,12.44%
test - fold3,14.40%,0.00%,0.00%,0.00%,35.20%,24.80%,12.80%,12.80%
train - fold4,19.47%,0.00%,0.00%,0.00%,36.62%,23.29%,8.36%,12.27%


In [117]:

train_id = train_fold[fold_num]
test_id = test_fold[fold_num]

In [118]:
before_test_path = os.path.join(_path, test_name)
train_path = os.path.join(_path, 'train')

for plane in os.listdir(before_test_path):
    if plane[0] == '.':
        continue
    print(plane)

    now_path = os.path.join(before_test_path, plane)
    des_path = os.path.join(train_path, plane)
    
    if not os.path.exists(des_path):
        os.makedirs(des_path)
    
    for num in tqdm(os.listdir(now_path)):
        shutil.move(
            os.path.join(now_path, num),
            os.path.join(des_path, num)
        )
if os.path.exists(os.path.join(_path, 'valid')):
    shutil.rmtree(os.path.join(_path, 'valid'))

sagittal


100%|██████████| 125/125 [00:00<00:00, 3555.70it/s]


axial


100%|██████████| 125/125 [00:00<00:00, 3182.60it/s]


coronal


100%|██████████| 125/125 [00:00<00:00, 3657.35it/s]


In [119]:
test_path = os.path.join(_path, 'test')
if not os.path.exists(test_path):
    os.makedirs(test_path)

for plane in os.listdir(train_path):

    if plane[0] == '.':
        continue
    print(plane)

    now_path = os.path.join(train_path, plane)
    des_test_path = os.path.join(test_path, plane)
    if not os.path.exists(des_test_path):
        os.makedirs(des_test_path)

    for num in tqdm(test_id):
        file_name = str(num).zfill(4) + '.npy'
        shutil.copy(
            os.path.join(now_path, file_name),
            os.path.join(des_test_path, file_name)
        )

sagittal


100%|██████████| 125/125 [00:00<00:00, 491.71it/s]


axial


100%|██████████| 125/125 [00:00<00:00, 471.20it/s]


coronal


100%|██████████| 125/125 [00:00<00:00, 522.94it/s]


In [120]:
csv_list = glob(os.path.join(_path, '*.csv'))
for csv_file in csv_list:
    os.remove(csv_file)

In [121]:
def name_change(x):
    return str(x).zfill(4)

train_line = [line for line in all if line[0] in train_id]
test_line = [line for line in all if line[0] in test_id]
print(len(train_line))
print(len(test_line))
train_csv = pd.DataFrame(train_line, columns=df.columns)
train_csv['name'] = train_csv['name'].apply(name_change)
valid_csv = pd.DataFrame(test_line, columns=df.columns)
valid_csv['name'] = valid_csv['name'].apply(name_change)

t_abnormal = train_csv[['name', 'abnormal']]
t_abnormal.to_csv(os.path.join(_path, "train-abnormal.csv"), header=False, index=False)
t_acl = train_csv[['name', 'acl']]
t_acl.to_csv(os.path.join(_path, "train-acl.csv"), header=False, index=False)
t_meniscus = train_csv[['name', 'meniscus']]
t_meniscus.to_csv(os.path.join(_path, "train-meniscus.csv"), header=False, index=False)

test_abnormal = valid_csv[['name', 'abnormal']]
test_abnormal.to_csv(os.path.join(_path, "test-abnormal.csv"), header=False, index=False)
test_acl = valid_csv[['name', 'acl']]
test_acl.to_csv(os.path.join(_path, "test-acl.csv"), header=False, index=False)
test_meniscus = valid_csv[['name', 'meniscus']]
test_meniscus.to_csv(os.path.join(_path, "test-meniscus.csv"), header=False, index=False)

1125
125
